In [1]:
import fio.database
import mip

db = fio.database.load_db("data")

In [2]:
import functools

def is_fluid(name):
    return db.fluid.query(f"""map(select(.name == "{name}"))|any""")

def is_item(name):
    return db.item.query(f"""map(select(.name == "{name}"))|any""")

assert is_item("iron-ore")
assert not is_fluid("iron-ore")

assert not is_item("water")
assert is_fluid("water")

@functools.lru_cache(16)
def get_assembly_machines(category, craftable_only=True):
  ms = []

  for m in db.assembling_machine.find(f""".[] | select(.crafting_categories."{category}")"""):
    if craftable_only and not db.recipe.query(f"""map( select(.main_product.name == "{m['name']}" )) | any"""):
      continue
    ms.append(m)

  for m in db.furnace.find(f""".[] | select(.crafting_categories."{category}") """):
    if craftable_only and not db.recipe.query(f"""map( select(.main_product.name == "{m['name']}" )) | any"""):
      continue
    ms.append(m)

  return ms

# assert get_assembly_machines("nuclear-fusion") == ['kr-fusion-reactor']
# assert set(get_assembly_machines("basic-crafting")) == {'assembling-machine-1', 'assembling-machine-2', 'assembling-machine-3', 'kr-advanced-assembling-machine'}

assert len(db.recipe.find(".[] | select(.hidden) | .name")) == 750
assert len(db.assembling_machine.find(""".[] | select(.crafting_categories."basic-crafting") | .name""")) == 6
assert db.recipe.query("""map( select(.main_product.name == "laser-turret" ))|any""") == True
assert db.recipe.query("""map( select(.main_product.name == "rien" ))|any""") == False

In [44]:
recipes = fio.database.HyperDiGraph()

ignore_machines = [
    "kr-advanced-assembling-machine",
    "kr-advanced-chemical-plant",
    "kr-advanced-furnace",
    "electric-furnace",
    "assembling-machine-3",
    "kr-matter-assembler",
    "kr-matter-plant",
    "kr-quantum-computer"
]

for r in db.recipe.find(".[]"):
    for m in get_assembly_machines(r['category']):

        if m["name"] in ignore_machines:
            continue

        A = {i["name"] for i in r["ingredients"]}
        B = {p["name"] for p in r["products"]}
        recipes.add_edge(A, B, {"recipe": r, "machine": m, "name": f"{r['name']}#{m['name']}"})


In [45]:
import fio.flow


source = {
    "iron-ore": float("+inf"),
    "crude-oil": float("+inf"),
    "copper-ore": float("+inf"),
    "stone": float("+inf"),
    "coal": float("+inf"),
    "water": float("+inf"),
    "wood": float("+inf"),
    "biomass": float("+inf"),
    "uranium-238": float("+inf"),
    "imersite-powder": float("+inf"),
    "mineral-water": float("+inf"),
    "raw-rare-metals": float("+inf"),
}

target = {
    k: 1 for k in {
        # 'basic-tech-card',
        # 'advanced-tech-card',
        'automation-science-pack',
        # 'biters-research-data',
        'chemical-science-pack',
        'logistic-science-pack',
        'production-science-pack',
        'utility-science-pack',
        # 'matter-research-data',
        # 'matter-tech-card',
        # 'military-science-pack',
        # 'singularity-tech-card',
    }
}

model = fio.flow.optimize_model(recipes, dict(**source, **{k: -v for k, v in target.items()}))

print(model["status"])
# model = build_detailed_graph(model)

OptimizationStatus.OPTIMAL


In [46]:

from collections import defaultdict

inputs = defaultdict(float)
outputs = defaultdict(float)
c = defaultdict(int)


flows = []
for r, m, k, f in [tuple(k.split("#") + list(v)) for k, v in model["flow"].items()]:
    c[m] += k

    m1 = db.assembling_machine.query(f"""map( select(.name == "{m}")) """)
    m2 = db.furnace.query(f"""map( select(.name == "{m}")) """)
    m = (m1 + m2)[0]
    r = db.recipe.query(f"""map( select(.name == "{r}"))""")[0]

    flows.append(r, m, k, f)
  
#     for i in r["ingredients"]:
#       inputs[i["name"]] += i["amount"] * m["crafting_speed"] * (i["probability"] if "probability" in i else 1) / r["energy"]
    
#     for i in r["products"]:
#       if "amount" in i:
#         amount = i["amount"]
#       elif "amount_min" in i and "amount_max" in i:
#         amount = (i["amount_min"] + i["amount_max"]) / 2
#       outputs[i["name"]] += amount * m["crafting_speed"] * (i["probability"] if "probability" in i else 1) / r["energy"]


# diff = {k: inputs[k] - outputs[k] for k in set(inputs.keys()) | set(outputs.keys())
# if inputs[k] - outputs[k] != 0}

# print(c)
# print(diff)

defaultdict(<class 'int'>, {'assembling-machine-2': 137, 'steel-furnace': 112, 'kr-atmospheric-condenser': 5, 'chemical-plant': 6, 'oil-refinery': 12, 'kr-crusher': 5, 'kr-research-server': 8, 'kr-filtration-plant': 4, 'kr-fuel-refinery': 16})
{'fast-transport-belt': -1.0, 'copper-ore': 1.25, 'light-oil': -7.125, 'electronic-components': 0.125, 'coke': -0.5, 'sand': -0.7380952380952381, 'sulfur': 3.0, 'processing-unit': 0.125, 'coal': 1.75, 'iron-gear-wheel': 7.6875, 'wood': 1.125, 'quartz': -0.6071428571428572, 'petroleum-gas': 39.0, 'iron-stick': -1.5, 'blank-tech-card': -0.3125, 'copper-plate': 3.5, 'electronic-circuit': 0.4375, 'logistic-science-pack': -0.1875, 'chemical-science-pack': -0.1875, 'oxygen': -5.875, 'automation-core': -0.7125, 'automation-science-pack': -0.1875, 'rocket-fuel': 0.1875, 'heavy-oil': -5.0, 'glass': -0.0625, 'iron-plate': 7.3125, 'steel-plate': -0.55, 'uranium-238': 0.25, 'raw-rare-metals': 1.25, 'advanced-circuit': 0.1875, 'iron-ore': 1.25, 'plastic-bar':

# Second step.


We now need to know how many machines are needed for which exchange. To do so, we build an other linear program, with pack bins everywhere.




# Bin packing

Inputs:
* $v[j]$ output flow required
* $u[i]$ input flow required

Variables:
* $x[i, j]$ real between 0 and 1, indicate the flow from $i$ to j$
* $y[i, j]$ boolean indicate if the edge from $i$ to $j$

Minimize $\displaystyle \sum_{i, j} y[i, j]$
Subject:
* $\displaystyle \forall i j, x[i, j] \leq y[i, j]$
* $\displaystyle \forall i, \sum_j x[i, j] = u_i$
* $\displaystyle \forall j, \sum_i x[i, j] = v_i$




In [ ]:
import fio.binpacking

fio.binpacking.optimize([0.1, 0.2, 0.3], [0.3, 0.2, 0.1])


In [ ]:
for n, info in model.nodes(data=True):
    if info["kind"] != "exchange":
        continue

    A = {(u, v): k for u, v, k in model.in_edges(n, data="flow")}
    B = {(u, v): k for u, v, k in model.out_edges(n, data="flow")}
    if sum(A.values()) < sum(B.values()):
        A["source", n] = sum(map(arith.float_to_frac, B.values())) - sum(map(arith.float_to_frac, A.values()))
    
    if sum(A.values()) > sum(B.values()):
        B["sink", n] = sum(map(arith.float_to_frac, A.values())) - sum(map(arith.float_to_frac, B.values()))

    m = arith.float_to_frac(sum(A.values()))
    A = {k: (arith.float_to_frac(v/m)) for k, v in A.items()}
    B = {k: (arith.float_to_frac(v/m)) for k, v in B.items()}


    print(n)
    print({k: arith.float_to_frac(v)*m for k, v in A.items()})
    print({k: arith.float_to_frac(v)*m for k, v in B.items()})
    print(fio.binpacking.optimize(A, B))
    print()